# Huffman Compress

Para testar o algoritmo de Huffman foram criados métodos para compactar e descompactar arquivos. 

O exemplo utilizado aqui é a compactação e descompactação de imagens do formato PPM e do formato PNG.

Foi utilizado a seguinte imagem para testar a aplicação:

![image](github.png)

In [1]:
import heapq
import os
from functools import total_ordering

In [2]:
@total_ordering
class HeapNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    # defining comparators less_than and equals
    def __lt__(self, other):
        return self.freq < other.freq

    def __eq__(self, other):
        if(other == None):
            return False
        if(not isinstance(other, HeapNode)):
            return False
        return self.freq == other.freq

In [3]:
class HuffmanCoding:
    def __init__(self, path):
        self.path = path
        self.heap = []
        self.codes = {}
        self.reverse_mapping = {}

    # functions for compression:

    def make_frequency_dict(self, text):
        frequency = {}
        for character in text:
            if not character in frequency:
                frequency[character] = 0
            frequency[character] += 1
        return frequency

    def make_heap(self, frequency):
        for key in frequency:
            node = HeapNode(key, frequency[key])
            heapq.heappush(self.heap, node)

    def merge_nodes(self):
        while(len(self.heap)>1):
            node1 = heapq.heappop(self.heap)
            node2 = heapq.heappop(self.heap)

            merged = HeapNode(None, node1.freq + node2.freq)
            merged.left = node1
            merged.right = node2

            heapq.heappush(self.heap, merged)


    def make_codes_helper(self, root, current_code):
        if(root == None):
            return

        if(root.char != None):
            self.codes[root.char] = current_code
            self.reverse_mapping[current_code] = root.char
            return

        self.make_codes_helper(root.left, current_code + "0")
        self.make_codes_helper(root.right, current_code + "1")


    def make_codes(self):
        root = heapq.heappop(self.heap)
        current_code = ""
        self.make_codes_helper(root, current_code)


    def get_encoded_text(self, text):
        encoded_text = ""
        for character in text:
            encoded_text += self.codes[character]
        return encoded_text


    def pad_encoded_text(self, encoded_text):
        extra_padding = 8 - len(encoded_text) % 8
        for i in range(extra_padding):
            encoded_text += "0"

        padded_info = "{0:08b}".format(extra_padding)
        encoded_text = padded_info + encoded_text
        return encoded_text


    def get_byte_array(self, padded_encoded_text):
        if(len(padded_encoded_text) % 8 != 0):
            print("Encoded text not padded properly")
            exit(0)

        b = bytearray()
        for i in range(0, len(padded_encoded_text), 8):
            byte = padded_encoded_text[i:i+8]
            b.append(int(byte, 2))
        return b


    def compress(self):
        filename, file_extension = os.path.splitext(self.path)
        output_path = filename + ".bin"

        with open(self.path, 'r+') as file, open(output_path, 'wb') as output:
            text = file.read()
            text = text.rstrip()

            frequency = self.make_frequency_dict(text)
            self.make_heap(frequency)
            self.merge_nodes()
            self.make_codes()

            encoded_text = self.get_encoded_text(text)
            padded_encoded_text = self.pad_encoded_text(encoded_text)

            b = self.get_byte_array(padded_encoded_text)
            output.write(bytes(b))

        print("Compressed")
        return output_path


    """ functions for decompression: """


    def remove_padding(self, padded_encoded_text):
        padded_info = padded_encoded_text[:8]
        extra_padding = int(padded_info, 2)

        padded_encoded_text = padded_encoded_text[8:] 
        encoded_text = padded_encoded_text[:-1*extra_padding]

        return encoded_text

    def decode_text(self, encoded_text):
        current_code = ""
        decoded_text = ""

        for bit in encoded_text:
            current_code += bit
            if(current_code in self.reverse_mapping):
                character = self.reverse_mapping[current_code]
                decoded_text += character
                current_code = ""

        return decoded_text


    def decompress(self, input_path):
        filename, file_extension = os.path.splitext(self.path)
        output_path = filename + "_decompressed" + ".ppm"

        with open(input_path, 'rb') as file, open(output_path, 'w') as output:
            bit_string = ""

            byte = file.read(1)
            while(len(byte) > 0):
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8, '0')
                bit_string += bits
                byte = file.read(1)

            encoded_text = self.remove_padding(bit_string)

            decompressed_text = self.decode_text(encoded_text)

            output.write(decompressed_text)

        print("Decompressed")
        return output_path


In [4]:
def convert_image_to_ppm(path):
    file_name = path.split('.')[0]
    os.system('convert ' + path + ' -compress none ' + file_name + '.ppm')
    return file_name + '.ppm'

In [5]:
def convert_image_to_png(path):
    file_name = path.split('.')[0]
    os.system('convert ' + file_name + '_decompressed.ppm' + ' -compress none ' + file_name + '_decompressed.png')
    os.system('rm ' + path)
    os.system('rm ' + file_name + '_decompressed.ppm')
    return 'Converted'

In [6]:
import sys

path = "lena.ppm"

h = HuffmanCoding(path)

In [7]:
output_path = h.compress()
print("Compressed file path: " + output_path)

Compressed
Compressed file path: lena.bin


In [8]:
decom_path = h.decompress(output_path)
print("Decompressed file path: " + decom_path)

Decompressed
Decompressed file path: lena_decompressed.ppm


In [9]:
print(h.heap)

[]


In [10]:
print(h.codes)

{'1': '111', '2': '000', '\n': '110001', '5': '0100', '9': '1101', ' ': '10', '8': '0110', '3': '0111', 'P': '110000', '7': '11001', '6': '0101', '4': '0010', '0': '0011'}


In [11]:
print(h.reverse_mapping)

{'0011': '0', '11001': '7', '10': ' ', '0111': '3', '111': '1', '0010': '4', '0100': '5', '000': '2', '0101': '6', '110001': '\n', '110000': 'P', '0110': '8', '1101': '9'}


In [12]:
path = 'github.png'

path = convert_image_to_ppm(path)

h = HuffmanCoding(path)

output_path = h.compress()
print("Compressed file path: " + output_path)

decom_path = h.decompress(output_path)
print("Decompressed file path: " + decom_path)

Compressed
Compressed file path: github.bin
Decompressed
Decompressed file path: github_decompressed.ppm


In [13]:
convert_image_to_png(path)

'Converted'